In [1]:
from src.BaseNetwork import BaseNetwork
from models.rnn import LSTM
import torch

config_path = 'configs/rnn.yaml'

model_constructor = LSTM 

In [2]:
import numpy as np
def generate_data(seq_length, num_samples=1000):
    x = np.linspace(0, 100, num_samples)
    y = np.sin(x) + 0.1 * np.random.normal(size=num_samples)
    data_x = []
    data_y = []
    for i in range(len(y) - seq_length):
        data_x.append(y[i:i+seq_length])
        data_y.append(y[i+seq_length])
    return np.array(data_x), np.array(data_y)


In [ ]:
# load your data, then convert it into torch.tensor. I suppose that your tensors will have shape like data=(time, n_features), target=(time, 1)
data, target = generate_data(30)
data = torch.tensor(data, dtype=torch.float32)
target = torch.tensor(target, dtype=torch.float32).unsqueeze(1)

In [6]:
target.shape

torch.Size([970, 1])

In [7]:
model_pipeline = BaseNetwork(
    config_path=config_path,
    model_class_constructor=model_constructor,
    data=data,
    target=target
)

In [8]:
loader = model_pipeline.train(return_model = False, return_loss_logs = True)

  0%|          | 0/56 [00:00<?, ?it/s]/Users/ilya/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Training loss: 0.15930211544036865:  45%|████▍     | 25/56 [00:00<00:00, 242.36it/s]/Users/ilya/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([12, 1, 1])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Training loss: 0.15312904119491577: 100%|██████████| 56/56 [00:00<00:00, 276.74it/s]
/Users/ilya/Library/Python/3.9/lib/python/site-packages/tor